In [1]:
%matplotlib inline
import os, sys, time
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from __future__ import print_function

In [2]:
# load the raw data
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
print('Shape of the training dataset: {}'.format(df_train.shape))
print('Shape of the test dataset: {}'.format(df_test.shape))
df_train.head()

Shape of the training dataset: (595212, 59)
Shape of the test dataset: (892816, 58)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [3]:
# is there class imbalance? The answer is very much...
df_train['target'].value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [4]:
# what features are we looking at?
# there are 57 features in total (=59-2)
all_features = df_train.columns.tolist()
all_features.remove('target')
all_features.remove('id')
bin_features = []
cat_features = []
other_features = []
for col in df_train.columns[2:]:  # exclude 'id' and 'target'
    if col.split('_')[-1] == 'bin':
        bin_features.append(col)
    elif col.split('_')[-1] == 'cat':
        cat_features.append(col)
    else:
        other_features.append(col)

In [5]:
len(bin_features)

17

In [6]:
len(cat_features)

14

In [7]:
# how many missing values are we looking at?
# percentage by classes
df_tmp = df_train[['target']+all_features].groupby('target')[all_features]\
.apply(lambda x: 100*np.sum(x == -1)/len(x)).T
# percentage by all classes
df_tmp2 = (df_train[all_features].apply(lambda x: x == -1).sum() / df_train.shape[0] * 100)\
.to_frame()
df_tmp2.rename(columns={0: 'total'}, inplace=True)
# merge!
df_tmp = df_tmp.merge(df_tmp2, left_index=True, right_index=True)
del df_tmp2
df_tmp.sort_values(by='total', ascending=False, inplace=True)
df_tmp.head(14)  # there are 13 features with missing values

,0,1,total
ps_car_03_cat,69.358067,61.998709,69.089837
ps_car_05_cat,45.002772,38.960081,44.782531
ps_reg_03,18.257840,14.105283,18.106490
ps_car_14,7.130901,7.942288,7.160474
ps_car_07_cat,1.846673,4.139393,1.930237
ps_ind_05_cat,0.928480,2.231032,0.975955
ps_car_09_cat,0.090145,0.239698,0.095596
ps_ind_02_cat,0.030688,0.184383,0.036290
ps_car_01_cat,0.012728,0.156725,0.017977
ps_ind_04_cat,0.008718,0.152116,0.013945


## Dealing with missing values
Before we dive deep into the model, we need to deal with the missing values first. The simpliest method is just to ignore all the samples that as **any** missing value in there. However, by doing so, we reduce the total sample size from ~ 600k to less than 100k. 

Therefore, we need a better way to impute the missing values. Jingfei's current methodolgy is to treat the missing value as a new category, for the cases of binary and categorical features. (How about continuous features?)

Another possibility is to geuss the missing values intelligently. To start, we can fill the missing binary and categorical values with the most frequency entries, and fill the missing continuous with their medians.

There are much more can be done with regard to data imputation: for example, to train regression models on the missing variable, from other varibles. However, we need to keep in mind how to carry the regression to test data or new data, with missing entries. 

We have to pay special attention to those features with many missing values, namely, `ps_car_03_cat`, `ps_cat_05_cat`, `ps_reg_03`, etc.

### Metric

When comparing different imputation methods, we need have a unified metric to evaluate their success. The simplest metric can be the testing error - same as how we evaluate different models, but in this case, the model should stay the same. The default choice will be xgboost.

In [8]:
# replace -1 with nan for easier handling
df_train.replace({-1: np.float('NaN')}, inplace=True)

# helper function to get the best and last training, valid gini coefficients
def get_gini_stats(eval_results):
    last_training, last_valid = [], []
    best_training, best_valid = [], []
    for x in eval_results:
        last_training.append(x['train']['gini'][-1])
        last_valid.append(x['valid']['gini'][-1])
        best_training.append(max(x['train']['gini']))
        best_valid.append(max(x['valid']['gini']))
    return last_training, last_valid, best_training, best_valid

### Method 1: mode and median
In the simplest case, let's fill the missing binary / categorical values with their mode, and continoues variables with their median.

In [9]:
# can also be done with sklearn
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html
fills = {}  # make the dict so that we can fill test data
for c in df_train.columns[2:]:
    if c.endswith(('bin', 'cat')):
        fills[c] = df_train[c].mode()[0]
    else: 
        fills[c] = df_train[c].median()
df_train_impute_1 = df_train.fillna(fills)
# df_test_impute_1 = df_test.fillna(fills)

In [10]:
%run '../py/models_ccy.py'
clf = my_xgb(df_train_impute_1[all_features], 
             df_train_impute_1['target'].astype(int, inplace=True))
clf.input_scaling(ratio=0.05)

Splitted training/test, and applied standard scaler.
Total number of training samples: 565451


In [11]:
# clf.one_hot_encoding()
models_impute_1, eval_results_impute_1 = clf.fit(one_hot=False)

Performing 5 fold CV


//anaconda/envs/python27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Fold 1 of 5
class weight is: 26
[0]	train-gini:0.229256	valid-gini:0.21061
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.386398	valid-gini:0.275273
[200]	train-gini:0.453984	valid-gini:0.272871
Stopping. Best iteration:
[137]	train-gini:0.412761	valid-gini:0.276752

Fold 2 of 5
class weight is: 26
[0]	train-gini:0.232125	valid-gini:0.202841
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.389445	valid-gini:0.266435
[200]	train-gini:0.461067	valid-gini:0.266343
Stopping. Best iteration:
[135]	train-gini:0.415507	valid-gini:0.268337

Fold 3 of 5
class weight is: 26
[0]	train-gini:0.233745	valid-gini:0.206239
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]

In [12]:
last_training, last_valid, best_training, best_valid = get_gini_stats(eval_results_impute_1)
print(np.mean(last_valid))
print(np.std(last_valid))

0.2729496
0.00683717446318


### Method 2: random imputation
The idea here is that, for the missing values, we want to fill them with values drawn randomly from their underlying distribution. We will use the existing values to infer the underlying distribution. This can be easily done for binary and categorical features. (How to do this for continous feature?)

In [13]:
# get distributions for a categorical feature
def fill_w_distribution(s):
    '''
    Input
    =======
    s: <pd series>
        with NaN as missing values. Binary or categorical feature
    Return
    =======
    filled_s: <pd series>
        without NaNs
    '''
    import numpy as np
    
    Xs = s.value_counts().index.values
    p  = s.value_counts().values
    p  = 1.0 * p / p.sum()
    num_to_fill = s.isnull().sum()
    idx_to_fill = s[s.isnull()].index
    fills = np.random.choice(Xs, size=num_to_fill, p=p)
    val_to_fill = pd.Series(fills, index=idx_to_fill)
    # fill it!
    filled_s = s.add(val_to_fill, fill_value=0)
    
    return filled_s

In [14]:
df_train_impute_2 = df_train.copy()
for c in df_train_impute_2.columns[2:]:
    if c.endswith(('bin', 'cat')):
        df_train_impute_2[c] = fill_w_distribution(df_train_impute_2[c])       
# fill the continous NaN with median 
df_train_impute_2 = df_train_impute_2.fillna(df_train_impute_2.median())

In [15]:
clf = my_xgb(df_train_impute_2[all_features], 
             df_train_impute_2['target'].astype(int, inplace=True))
clf.input_scaling(ratio=0.05)

Splitted training/test, and applied standard scaler.
Total number of training samples: 565451


In [16]:
# clf.one_hot_encoding()
models_impute_2, eval_results_impute_2 = clf.fit(one_hot=False)

Performing 5 fold CV
Fold 1 of 5
class weight is: 26
[0]	train-gini:0.226659	valid-gini:0.199199
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.385047	valid-gini:0.270573
[200]	train-gini:0.459176	valid-gini:0.271141
Stopping. Best iteration:
[151]	train-gini:0.424283	valid-gini:0.272074

Fold 2 of 5
class weight is: 26
[0]	train-gini:0.230289	valid-gini:0.203716
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.388984	valid-gini:0.275365
[200]	train-gini:0.459629	valid-gini:0.274149
Stopping. Best iteration:
[142]	train-gini:0.420898	valid-gini:0.276718

Fold 3 of 5
class weight is: 26
[0]	train-gini:0.230899	valid-gini:0.200549
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improve

In [17]:
last_training, last_valid, best_training, best_valid = get_gini_stats(eval_results_impute_2)
print(np.mean(last_valid))
print(np.std(last_valid))

0.2701592
0.00268336008765


### Method 3: imputation with joint distribution
The idea is that, we want to guess the missing values of one feature, with the joint distribution bewteen this feature and other features. In another word, we want to train a simple model to predict the missing values. A possible model can be Naive Bayes with Gaussian distribution. The choice of such model is that there is no hyper parameter, hence we can just keep things simple (to start with).

In [18]:
feature_wo_nan, feature_w_nan = [], []
for c in df_train.iloc[:, 2:]:
    if df_train[c].isnull().sum() == 0:
        feature_wo_nan.append(c)
    else:
        feature_w_nan.append(c)

def fill_w_NB(df_train, col, feature_wo_na):
    '''
    Input
    =========
    df_train: <pd dataframe>
        dataframe with the NaN features
    col: <str>
        name of the column to fit
    feature_wo_nan: <list>
        list of column names without NaN features
        
    Return
    =========
    filled_s: <pd Series>
        pandas series without NaN
    gnb: <>
    '''
    
    # get the 'training' and 'test' set
    df_tmp = df_train.iloc[:, 2:]
    df_train_tmp = df_tmp[df_tmp[col].notnull()]
    df_test_tmp = df_tmp[df_tmp[col].isnull()]
    df_test_tmp.drop(col, axis=1, inplace=True)
    # train a NB model
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(df_train_tmp[feature_wo_nan], df_train_tmp[col])
    fills = gnb.predict(df_test_tmp[feature_wo_nan])
    val_to_fill = pd.Series(fills, index=df_test_tmp.index)
    filled_s = df_tmp[col].add(val_to_fill, fill_value=0)
    
    return filled_s, gnb

In [19]:
impute_3_models = {}
df_train_impute_3 = df_train.copy()
for f in feature_w_nan:
    if f.endswith(('cat', 'bin')):
        filled_s, model = fill_w_NB(df_train, f, feature_wo_nan)
        df_train_impute_3[f] = filled_s
        impute_3_models[f] = model
# fill continouse features with median
df_train_impute_3 = df_train_impute_3.fillna(df_train_impute_3.median()) 

/anaconda/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
clf = my_xgb(df_train_impute_3[all_features], 
             df_train_impute_3['target'].astype(int, inplace=True))
clf.input_scaling(ratio=0.05)

Splitted training/test, and applied standard scaler.
Total number of training samples: 565451


In [21]:
# clf.one_hot_encoding()
models_impute_3, eval_results_impute_3 = clf.fit(one_hot=False)

Performing 5 fold CV
Fold 1 of 5
class weight is: 26
[0]	train-gini:0.232632	valid-gini:0.200026
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.38412	valid-gini:0.278988
[200]	train-gini:0.458704	valid-gini:0.279598
Stopping. Best iteration:
[169]	train-gini:0.436914	valid-gini:0.281969

Fold 2 of 5
class weight is: 26
[0]	train-gini:0.233138	valid-gini:0.186181
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.3925	valid-gini:0.261193
[200]	train-gini:0.464098	valid-gini:0.25996
Stopping. Best iteration:
[151]	train-gini:0.431637	valid-gini:0.262414

Fold 3 of 5
class weight is: 26
[0]	train-gini:0.225105	valid-gini:0.210068
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in

In [28]:
last_training, last_valid, best_training, best_valid = get_gini_stats(eval_results_impute_3)
print(np.mean(last_valid))
print(np.std(last_valid))

0.2737044
0.00881330388901


## Feature engineering

In [23]:
# TODO